# Ordre de travail




In [1]:
import pandas as pd


In [3]:
df=pd.read_csv("C:/Users/imami/OneDrive/Bureau/projet fil rouge/Projet_Fil_Rouge_Telecom_Diag/Projet_Fil_Rouge_Telecom_Diag/Data/OT/ot_data_V6_douai_v2.csv")

In [4]:
df.columns

Index(['Unnamed: 0', 'ACTIVITES', 'CD_ART', 'CHG_REL', 'COMMENT_OT', 'DES_EFP',
       'DES_EQT', 'DES_FP', 'DES_LIG', 'DES_MAT', 'DES_OP', 'DES_PT',
       'DES_SFP', 'D_REF_OT', 'DUR', 'FLAG_ACTIVITES', 'LIB_C_COUT', 'LIB_FAM',
       'LIB_OT', 'LIB_TYP_W', 'NOM_FOUR', 'NUM_OT', 'NUM_OT_PERE',
       'OBJ_REF_EQT', 'OBJ_REF_PT', 'REF_FOUR', 'ZONE_TRI', 'Full_Text'],
      dtype='object')

In [5]:
#traitement colonne ACTIVITIES
#the column always starts with "Description de l’activité attendue et/ou explication du problème :  ", we'll remove it
l=len('Description de l’activité attendue et/ou explication du problème :')
df['ACTIVITES']=df['ACTIVITES'].str[l+1:]

In [6]:
df[df['ACTIVITES']!='']['ACTIVITES'].count()

2971

la colonne CD_ART est vide, la colonne CHG_REL est une colonne numérique reste à comprendre sa signification, on passe à la colonne COMMENT_OD

In [7]:
#traitement colonne COMMENT_OT
l=len("Commentaire de l’ordre de travail : ")
df['COMMENT_OT']=df['COMMENT_OT'].str[l:]



Le code utiliser pour parser et avoir chaque symptome cause solution dans sa propre colonne respectivement

In [8]:
import re

# Input text
text = """23.01.2023 08:50:20 A______ :    . Symptôme: NOMBREUX DEFAUT SERTISSAGES SUR LA TETE 2 RIVKLE    . Cause: SURPRESSEUR HS    . Solution: REPRISE DES POINTS DE DEPOSE RIVKLE    NOK   REFAIT ETALONNAGE CAPTEUR COURSE DE SERTISSAGE    NOK REM PLACEMENT DE LA TETE DE SERTISSAGE NOK   REMPLACEMENT DU SURPRESSEUR PAR L'EQUIPE DE NUIT    OK    . Validation: OK"""

# Regular expression pattern
pattern = r"""
    . Symptôme: (?P<symptome>.*?)    \.     # Match "Symptôme" section
    . Cause: (?P<cause>.*?)    \.         # Match "Cause" section
    . Solution: (?P<solution>.*?)    \.   # Match "Solution" section
    . Validation: (?P<validation>.*)      # Match "Validation" section
"""

# Compile the regex pattern with verbose flag
regex = re.compile(pattern, re.VERBOSE)

# Apply regex to find the groups
match = regex.search(text)
if match:
    symptome = match.group("symptome")
    cause = match.group("cause")
    solution = match.group("solution")
    validation = match.group("validation")

    # Print the results
    print("Symptôme:", symptome)
    print("Cause:", cause)
    print("Solution:", solution)
    print("Validation:", validation)
else:
    print("No match found!")

Symptôme:  NOMBREUX DEFAUT SERTISSAGES SUR LA TETE 2 RIVKLE    
Cause:  SURPRESSEUR HS    
Solution:  REPRISE DES POINTS DE DEPOSE RIVKLE    NOK   REFAIT ETALONNAGE CAPTEUR COURSE DE SERTISSAGE    NOK REM PLACEMENT DE LA TETE DE SERTISSAGE NOK   REMPLACEMENT DU SURPRESSEUR PAR L'EQUIPE DE NUIT    OK    
Validation:  OK


In [9]:
symptome_list=[]
cause_list=[]
solution_list=[]
validation_list=[]

for text in df['COMMENT_OT']:
  match = regex.search(text)
  if match:
    symptome = match.group("symptome")
    cause = match.group("cause")
    solution = match.group("solution")
    validation = match.group("validation")

    #append the result
    symptome_list.append(symptome)
    cause_list.append(cause)
    solution_list.append(solution)
    validation_list.append(validation)

ot_data={
    'symptome':symptome_list,
    'cause':cause_list,
    'solution':solution_list,
    'validation': validation
}
df_ot = pd.DataFrame(ot_data)

In [10]:
df_ot.count()

symptome      2827
cause         2827
solution      2827
validation    2827
dtype: int64

In [11]:
df_ot.head()

,symptome,cause,solution,validation
0,"ROBOT EN POSITION DE SOUDURE, DEFAUT ROBOT EN...",PRESENCE DE LIQUIDE DE REFROIDISSEMENT ENTRE ...,REMPLACEMENT DU FAISCEAU PLUS PASSAGE DU FIL ...,NON ANALYSE GATM
1,DEFAUT DE QUALITE DE SOUDURE,DEBIT EAU INSUFFISANT 5.15L,OT NUIT LE 11/03 POUR VERIF BOUCHON D'EAU NOK...,NON ANALYSE GATM
2,DEFAUT COLLE,AIR DANS LE CIRCUIT,PURGE POMPE 1 ET CIRCUIT NETTOYAGE BUSE ESSAI...,NON ANALYSE GATM
3,ARRET D'URGENCE FS3,PAS DE CONNEXION AVEC AUTOMATE,ECHANGE CARTE PMCIA ECHANGE CARTE UC ECHA...,NON ANALYSE GATM
4,DEFAUT L'AXE 2 N'ATTEINT PAS SON ZERO (E8.5 1),"CASSE LORS DE L'AUTRE EQUIPE, OUTIL COINCE DAN...",ATTENTE PIECE COMMANDER LE VENDREDI 16/06/2023,NON ANALYSE GATM


# Document pdf doc-R-30iB

In [12]:
%pip install PyPDF2 PyMuPDF pdfplumber

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\imami\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [13]:
import fitz  # PyMuPDF
import pdfplumber
from PyPDF2 import PdfWriter, PdfReader

In [14]:
#parse the error codes in the pdf and put it as plain text for preprocessing
pages_to_keep=[i for i in range(48,50)]

with pdfplumber.open("C:/Users/imami/OneDrive/Bureau/projet fil rouge/Projet_Fil_Rouge_Telecom_Diag/Projet_Fil_Rouge_Telecom_Diag/Data/Doc_Fournisseur/doc-R-30iB.pdf") as pdf:
        writer = PdfWriter()
        for page_num in pages_to_keep:
            # Get the page
            page = pdf.pages[page_num - 1]  # Pages are 0-indexed in pdfplumber

            # Extract text and remove header/footer
            # Assuming the first 10% and last 10% of the page are headers/footers
            cropped_text = page.within_bbox((0, page.height * 0.05, page.width, page.height * 0.95)).extract_text()
            print(cropped_text)

3.1 A
3.1.1 ACAL Alarm Code
3.1.1.1 ACAL-000 Fail to run DETECT Motn
Cause: TheAccuCal2Motionfailedtorun. TheAccuCal2motioncannotbestarted.
Remedy: Clear all the errors before executing AccuCal2.
3.1.1.2 ACAL-001 Robot is not ready.
Cause: Therobotisnotready. Thesystemcannotissuemotionbecauseitisinanerrorstate.
Remedy: Clear all faults, then retry the operation.
3.1.1.3 ACAL-002 Fail to fit circle.
Cause: AccuCal2cannotconvergewithinthesetthreshold.
Remedy: Perform the following:
• Increase the number of iterations allowed.
• Increase the convergence threshold setting.
• Make sure the TCP has not been changed.
3.1.1.4 ACAL-003 Contact before search.
Cause: Therobotisincontactwiththepartbeforestartingasearchmotion.
Remedy: Reteach the starting position.
3.1.1.5 ACAL-004 No contact detected.
Cause: NocontactwasmadeduringtheAccuCal2motion.
Remedy: Makesurethesensorandsoftwareareinstalledcorrectly.
3.1.1.6 ACAL-005 Undefined program name.
Cause: The AccuCal2 program name was not defined.
R

In [15]:
raw_text=''
pages_to_keep=[i for i in range(48,1976)]
with pdfplumber.open("C:/Users/imami/OneDrive/Bureau/projet fil rouge/Projet_Fil_Rouge_Telecom_Diag/Projet_Fil_Rouge_Telecom_Diag/Data/Doc_Fournisseur/doc-R-30iB.pdf") as pdf:
        writer = PdfWriter()
        for page_num in pages_to_keep:
            # Get the page
            page = pdf.pages[page_num - 1]  # Pages are 0-indexed in pdfplumber

            # Extract text and remove header/footer
            # Assuming the first 10% and last 10% of the page are headers/footers
            cropped_text = page.within_bbox((0, page.height * 0.05, page.width, page.height * 0.95)).extract_text()
            raw_text=raw_text+cropped_text


In [16]:
f = open("C:/Users/imami/OneDrive/Bureau/projet fil rouge/Projet_Fil_Rouge_Telecom_Diag/Projet_Fil_Rouge_Telecom_Diag/Data/Doc_Fournisseur/raw_text.txt", "a",  encoding="utf-8")
f.write(raw_text)
f.close()

In [17]:

symptome_list=[]
cause_list=[]
solution_list=[]

pattern = r"(?P<title>\d+\.\d+\.\d+\.\d+ [^\n]+)\s*Cause:\s*(?P<cause>[^R]+)\s*Remedy:\s*(?P<remedy>[^\n]+(?:\n\s*•[^\n]+)*)"

matches = re.finditer(pattern, raw_text)

for match in matches:
    symptome_list.append(match.group("title"))
    cause_list.append(match.group("cause").strip())
    solution_list.append(match.group("remedy").strip())

doc_data={
    'symptome':symptome_list,
    'cause':cause_list,
    'solution':solution_list
}
df_doc = pd.DataFrame(doc_data)

In [18]:
df_doc.head()

,symptome,cause,solution
0,3.1.1.1 ACAL-000 Fail to run DETECT Motn,TheAccuCal2Motionfailedtorun. TheAccuCal2motio...,Clear all the errors before executing AccuCal2.
1,3.1.1.2 ACAL-001 Robot is not ready.,Therobotisnotready. Thesystemcannotissuemotion...,"Clear all faults, then retry the operation."
2,3.1.1.3 ACAL-002 Fail to fit circle.,AccuCal2cannotconvergewithinthesetthreshold.,Perform the following:\n• Increase the number ...
3,3.1.1.4 ACAL-003 Contact before search.,Therobotisincontactwiththepartbeforestartingas...,Reteach the starting position.
4,3.1.1.5 ACAL-004 No contact detected.,NocontactwasmadeduringtheAccuCal2motion.,Makesurethesensorandsoftwareareinstalledcorrec...


In [19]:
df_doc.count()

symptome    7748
cause       7748
solution    7748
dtype: int64

Now we are going to combine the two dataframes (will drop the validation column from the first dataframe) 

In [20]:
df_ot_v1 = df_ot.drop('validation', axis=1)
df_total=pd.concat([df_ot_v1,df_doc],axis=0)

In [21]:
df_total.head()

,symptome,cause,solution
0,"ROBOT EN POSITION DE SOUDURE, DEFAUT ROBOT EN...",PRESENCE DE LIQUIDE DE REFROIDISSEMENT ENTRE ...,REMPLACEMENT DU FAISCEAU PLUS PASSAGE DU FIL ...
1,DEFAUT DE QUALITE DE SOUDURE,DEBIT EAU INSUFFISANT 5.15L,OT NUIT LE 11/03 POUR VERIF BOUCHON D'EAU NOK...
2,DEFAUT COLLE,AIR DANS LE CIRCUIT,PURGE POMPE 1 ET CIRCUIT NETTOYAGE BUSE ESSAI...
3,ARRET D'URGENCE FS3,PAS DE CONNEXION AVEC AUTOMATE,ECHANGE CARTE PMCIA ECHANGE CARTE UC ECHA...
4,DEFAUT L'AXE 2 N'ATTEINT PAS SON ZERO (E8.5 1),"CASSE LORS DE L'AUTRE EQUIPE, OUTIL COINCE DAN...",ATTENTE PIECE COMMANDER LE VENDREDI 16/06/2023


In [22]:
df_total.count()

symptome    10575
cause       10575
solution    10575
dtype: int64

# Generate a knowledge graph


## Problématique

Le fait que notre dataset de triplets `(symptôme, cause, solution)` soit généré à partir du **parsing de documents fournisseurs** et des **ordres de travail** entraîne un problème :  

- Les noeuds du graphe sont **uniques** et isolés.  
- En conséquence, le graphe de connaissance que nous construirons sera un **graphe sparse** avec **peu d’interconnexions** entre les entités.  

---

## Solutions pour Remédier à la Sparsité

1. **Utiliser des mesures de similarité sémantique** :  
   - Identifier et combiner les noeuds qui présentent un certain **overlap** dans leurs significations ou contenus.
   - Par exemple :  
     - Noeuds "Connexion lente" et "Internet lent" peuvent être fusionnés en un noeud commun "Problème de réseau".

2. **Appliquer un modèle de clustering** :  
   - Grouper les noeuds en **clusters** pour rassembler des entités similaires.
   - Par exemple :  
     - Problèmes "Connexion instable" et "Wi-Fi intermittent" peuvent être regroupés sous un cluster de type "Problèmes réseau".

3. **Utiliser des algorithmes de détection de communautés** :  
   - Identifier les **relations implicites** ou **groupes d'entités fortement connectées** dans le graphe.  
---

Ces approches permettront d’enrichir et de densifier le graphe, le rendant **moins sparse** et **plus interconnecté**, facilitant ainsi les analyses et les requêtes.  

In [2]:
from neo4j import GraphDatabase
uri = "bolt://localhost:7687"


ModuleNotFoundError: No module named 'neo4j'